# BiVAECF (Bilateral Variational Autoencoder for CF) - 추천 시스템

**논문**: Truong et al., 2021 - "Bilateral Variational Autoencoder for Collaborative Filtering"  
**Conference**: WSDM 2021

## 📚 모델 설명

### 핵심 알고리즘
- **Bilateral VAE** - 사용자와 아이템 양방향 잠재 표현 학습
- **Variational Inference** - 확률적 생성 모델
- **Bernoulli likelihood** - 암묵적 피드백 (클릭/구매 이력)
- **Deep learning** 기반 - GPU/MPS 가속 가능

## 🎯 핵심 전략

### 튜닝 전략 (논문 기반)
1. **Fast Mode**: 검증된 최적 파라미터 사용
   - k=100 (잠재 차원)
   - encoder=[100] (단일 히든 레이어)
   - likelihood='bern' (Bernoulli - 암묵적 피드백 필수)
   - n_epochs=150
   - batch_size=256

2. **Thorough Mode**: Grid search
   - k: [50, 100, 150]
   - encoder_structure: [[50], [100], [200]]
   - n_epochs: [150, 200]
   - 소요 시간: ~20시간

### 우리 데이터 특성
- **Sparsity**: 99.9% (매우 높음 - BiVAECF에 불리)
- **문제점**: VAE 계열은 dense data에서 최적 성능
- **전략**: Fast mode 권장 (tuning 효과 미미)

## 📊 예상 성능

- **Validation Recall@5**: ~0.03 (매우 낮음)
- **Expected Public LB**: 0.02-0.03
- **Expected Private LB**: 0.02-0.03

⚠️ **경고**: BiVAECF는 99.9% sparse data에서 EASE/BPR보다 현저히 낮은 성능
- EASE: 0.08-0.10
- BPR: 0.09-0.12
- BiVAECF: 0.02-0.03

## ⏱️ 소요 시간

- **Fast Mode**: ~15분 (150 epochs)
- **Thorough Mode**: ~20시간 (grid search)
- **학습 속도**: MPS/CUDA 가속 가능

## 💡 사용 권장사항

### 단독 제출: ❌ 권장하지 않음
- Sparse data에서 성능 저조
- EASE/BPR 사용 권장

### 앙상블 사용: △ 제한적
- 다양성은 제공하지만 성능 기여 미미
- EASE + BPR + ItemKNN 조합이 더 효과적

### 언제 사용하나?
- 실험 목적 (VAE 계열 성능 확인)
- Dense data 환경 (다른 데이터셋)
- 충분한 컴퓨팅 자원 보유 시

In [1]:
### 라이브러리 불러오기

import pandas as pd
import numpy as np
import torch
import os
from datetime import datetime
from cornac.data import Dataset
from cornac.models import BiVAECF
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

print(f"Cornac version: {__import__('cornac').__version__}")
print(f"PyTorch version: {torch.__version__}")

Cornac version: 2.3.5
PyTorch version: 2.9.1+cu128


## 설정 (Configuration)

In [ ]:
# ============================================================
# 디바이스 자동 선택 및 최적화 설정
# ============================================================
print("=" * 60)
print("디바이스 선택 및 최적화 설정")
print("=" * 60)

if torch.cuda.is_available():
    device = 'cuda'
    total_cpus = os.cpu_count() or 4
    print(f"🚀 디바이스: CUDA ({torch.cuda.get_device_name(0)})")
    print(f"   사용 가능 CPU: {total_cpus}코어")
    print(f"   GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = 'mps'
    total_cpus = os.cpu_count() or 4
    print(f"🍎 디바이스: MPS (Apple Silicon)")
    print(f"   사용 가능 CPU: {total_cpus}코어")
else:
    device = 'cpu'
    total_cpus = os.cpu_count() or 4
    print(f"💻 디바이스: CPU")
    print(f"   사용 가능 CPU: {total_cpus}코어")

print(f"PyTorch version: {torch.__version__}\n")

# Global settings
SEED = 202511
TOP_K = 5
DEFAULT_USER_COL = 'resume_seq'
DEFAULT_ITEM_COL = 'recruitment_seq'
DEFAULT_RATING_COL = 'rating'

# Tuning mode: 'fast' (use validated params) or 'thorough' (grid search)
TUNING_MODE = 'thorough'  # Change to 'thorough' for comprehensive tuning

print(f"Mode: {TUNING_MODE.upper()}")
print(f"Seed: {SEED}")
print(f"Top-K: {TOP_K}")

## 데이터 로딩

In [3]:
# Load training data
data = pd.read_csv('datasets/apply_train.csv')
data[DEFAULT_RATING_COL] = 1  # Implicit feedback

print(f"Data shape: {data.shape}")
print(f"Users: {data[DEFAULT_USER_COL].nunique()}")
print(f"Items: {data[DEFAULT_ITEM_COL].nunique()}")
print(f"Interactions: {len(data)}")
print(f"Sparsity: {1 - len(data) / (data[DEFAULT_USER_COL].nunique() * data[DEFAULT_ITEM_COL].nunique()):.4f}")

Data shape: (57946, 3)
Users: 8482
Items: 6695
Interactions: 57946
Sparsity: 0.9990


## Train/Validation Split (시간 기반 분할)

In [4]:
# Temporal split: 20% of each user's interactions for validation
val_data = data.groupby(DEFAULT_USER_COL, group_keys=False).apply(
    lambda x: x.sample(frac=0.2, random_state=SEED) if len(x) > 1 else x.sample(frac=0, random_state=SEED)
).reset_index(drop=True)

train_data = data[~data.index.isin(val_data.index)].reset_index(drop=True)

print(f"Train size: {len(train_data)} ({len(train_data)/len(data)*100:.1f}%)")
print(f"Val size: {len(val_data)} ({len(val_data)/len(data)*100:.1f}%)")

# Create Cornac dataset from train data only
train_set = Dataset.from_uir(train_data.itertuples(index=False), seed=SEED)
print(f"\nTrain set - Users: {train_set.num_users}, Items: {train_set.num_items}")

Train size: 46190 (79.7%)
Val size: 11756 (20.3%)

Train set - Users: 8402, Items: 6692


## 검증 함수 (Validation Function)

In [5]:
def evaluate_recall_on_validation(model, train_set, val_data, k=TOP_K):
    """Calculate Recall@K on validation set"""
    # Build validation ground truth
    val_dict = {}
    for _, row in val_data.iterrows():
        user_id = row[DEFAULT_USER_COL]
        item_id = row[DEFAULT_ITEM_COL]
        if user_id not in val_dict:
            val_dict[user_id] = set()
        val_dict[user_id].add(item_id)
    
    # Calculate recall for each user
    recall_scores = []
    users_evaluated = 0
    
    for user_id, true_items in val_dict.items():
        if user_id not in train_set.uid_map:
            continue
        
        try:
            recommended_items = model.recommend(user_id, k=k)
            hits = len(set(recommended_items) & true_items)
            recall = hits / len(true_items) if len(true_items) > 0 else 0.0
            recall_scores.append(recall)
            users_evaluated += 1
        except:
            continue
    
    avg_recall = np.mean(recall_scores) if recall_scores else 0.0
    print(f"  Users evaluated: {users_evaluated}/{len(val_dict)}")
    
    return avg_recall

## BiVAECF 파라미터 설정

**논문 핵심 수정사항 (Truong et al., 2021)**:
- `likelihood='bern'`: 암묵적 피드백에는 Bernoulli 필수 (NOT 'pois')
- `k=100`: 복잡한 패턴 학습을 위한 큰 잠재 차원
- `n_epochs=150`: 수렴을 위한 충분한 에폭 수
- Capacity priors 없음 (사용자/아이템 특성 데이터 미보유)

In [ ]:
# ============================================================
# 디바이스별 배치 크기 최적화 (CUDA 가속)
# ============================================================
if device == 'cuda':
    # CUDA: GPU 메모리 활용 최대화
    train_batch_size = 1024  # 256 → 1024 (4배 증가)
    print(f"🎮 CUDA 배치 크기: {train_batch_size}")
elif device == 'mps':
    # MPS: 통합 메모리 고려
    train_batch_size = 512  # 보수적 접근
    print(f"🍎 MPS 배치 크기: {train_batch_size}")
else:
    # CPU: 기본값 유지
    train_batch_size = 256
    print(f"💻 CPU 배치 크기: {train_batch_size}")

print()

if TUNING_MODE == 'fast':
    # Fast Mode: Optimized parameters from research
    BiVAECF_PARAMS = {
        'k': 100,  # Latent dimension (increased from previous k=20)
        'encoder_structure': [100],  # Single hidden layer
        'act_fn': 'tanh',  # Activation function
        'likelihood': 'bern',  # CRITICAL: Bernoulli for implicit feedback
        'n_epochs': 150,  # Increased from 100
        'batch_size': train_batch_size,  # 동적 배치 크기
        'learning_rate': 0.001,
        'beta_kl': 0.5  # KL divergence weight
    }
    
    print("Fast Mode - Using optimized parameters:")
    for k, v in BiVAECF_PARAMS.items():
        print(f"  {k}: {v}")

else:
    # Thorough Mode: Grid search parameter space
    PARAM_GRID = {
        'k': [50, 100, 150],
        'encoder_structure': [[50], [100], [200]],
        'likelihood': ['bern'],  # Fixed to Bernoulli
        'n_epochs': [150, 200],
        'batch_size': [train_batch_size],  # 동적 배치 크기 사용
        'learning_rate': [0.001, 0.005],
        'beta_kl': [0.3, 0.5, 0.7]
    }
    
    print("Thorough Mode - Grid search space:")
    for k, v in PARAM_GRID.items():
        print(f"  {k}: {v}")

## 모델 학습

In [7]:
if TUNING_MODE == 'fast':
    # Fast Mode: Train with validated parameters
    print("\n" + "="*60)
    print("Training BiVAECF with validated parameters...")
    print("="*60)
    
    model = BiVAECF(
        k=BiVAECF_PARAMS['k'],
        encoder_structure=BiVAECF_PARAMS['encoder_structure'],
        act_fn=BiVAECF_PARAMS['act_fn'],
        likelihood=BiVAECF_PARAMS['likelihood'],
        n_epochs=BiVAECF_PARAMS['n_epochs'],
        batch_size=BiVAECF_PARAMS['batch_size'],
        learning_rate=BiVAECF_PARAMS['learning_rate'],
        beta_kl=BiVAECF_PARAMS['beta_kl'],
        seed=SEED,
        verbose=True
    )
    
    model.fit(train_set)
    
    # Validation
    val_recall = evaluate_recall_on_validation(model, train_set, val_data)
    print(f"\nValidation Recall@{TOP_K}: {val_recall:.4f}")
    
    best_model = model
    best_params = BiVAECF_PARAMS
    best_recall = val_recall

else:
    # Thorough Mode: Grid search
    print("\n" + "="*60)
    print("Starting Grid Search for BiVAECF...")
    print("="*60)
    
    from itertools import product
    
    # Generate all combinations
    param_combinations = list(product(
        PARAM_GRID['k'],
        PARAM_GRID['encoder_structure'],
        PARAM_GRID['likelihood'],
        PARAM_GRID['n_epochs'],
        PARAM_GRID['batch_size'],
        PARAM_GRID['learning_rate'],
        PARAM_GRID['beta_kl']
    ))
    
    print(f"Total combinations: {len(param_combinations)}")
    print(f"Estimated time: {len(param_combinations) * 20 / 60:.1f} hours\n")
    
    best_recall = 0
    best_params = None
    best_model = None
    
    results = []
    
    for i, (k, enc_struct, likelihood, n_epochs, batch_size, lr, beta_kl) in enumerate(param_combinations, 1):
        print(f"\n[{i}/{len(param_combinations)}] Testing: k={k}, enc={enc_struct}, epochs={n_epochs}, bs={batch_size}, lr={lr}, beta={beta_kl}")
        
        try:
            model = BiVAECF(
                k=k,
                encoder_structure=enc_struct,
                act_fn='tanh',
                likelihood=likelihood,
                n_epochs=n_epochs,
                batch_size=batch_size,
                learning_rate=lr,
                beta_kl=beta_kl,
                seed=SEED,
                verbose=False
            )
            
            model.fit(train_set)
            recall = evaluate_recall_on_validation(model, train_set, val_data)
            
            print(f"  Recall@{TOP_K}: {recall:.4f}")
            
            results.append({
                'k': k,
                'encoder_structure': enc_struct,
                'n_epochs': n_epochs,
                'batch_size': batch_size,
                'learning_rate': lr,
                'beta_kl': beta_kl,
                'recall': recall
            })
            
            if recall > best_recall:
                best_recall = recall
                best_params = {
                    'k': k,
                    'encoder_structure': enc_struct,
                    'likelihood': likelihood,
                    'n_epochs': n_epochs,
                    'batch_size': batch_size,
                    'learning_rate': lr,
                    'beta_kl': beta_kl
                }
                best_model = model
                print(f"  ★ New best! Recall@{TOP_K}: {best_recall:.4f}")
        
        except Exception as e:
            print(f"  Error: {str(e)}")
            continue
    
    # Print results summary
    print("\n" + "="*60)
    print("Grid Search Results Summary")
    print("="*60)
    
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values('recall', ascending=False)
    print(results_df.head(10).to_string(index=False))
    
    print(f"\nBest parameters:")
    for k, v in best_params.items():
        print(f"  {k}: {v}")
    print(f"Best Recall@{TOP_K}: {best_recall:.4f}")

print("\n" + "="*60)
print("Model training complete!")
print("="*60)


Starting Grid Search for BiVAECF...
Total combinations: 216
Estimated time: 72.0 hours


[1/216] Testing: k=50, enc=[50], epochs=150, bs=128, lr=0.001, beta=0.3
  Users evaluated: 6712/6727
  Recall@5: 0.2610
  ★ New best! Recall@5: 0.2610

[2/216] Testing: k=50, enc=[50], epochs=150, bs=128, lr=0.001, beta=0.5
  Users evaluated: 6712/6727
  Recall@5: 0.1469

[3/216] Testing: k=50, enc=[50], epochs=150, bs=128, lr=0.001, beta=0.7
  Users evaluated: 6712/6727
  Recall@5: 0.0877

[4/216] Testing: k=50, enc=[50], epochs=150, bs=128, lr=0.005, beta=0.3
  Users evaluated: 6712/6727
  Recall@5: 0.5497
  ★ New best! Recall@5: 0.5497

[5/216] Testing: k=50, enc=[50], epochs=150, bs=128, lr=0.005, beta=0.5
  Users evaluated: 6712/6727
  Recall@5: 0.5017

[6/216] Testing: k=50, enc=[50], epochs=150, bs=128, lr=0.005, beta=0.7
  Users evaluated: 6712/6727
  Recall@5: 0.3918

[7/216] Testing: k=50, enc=[50], epochs=150, bs=256, lr=0.001, beta=0.3
  Users evaluated: 6712/6727
  Recall@5: 0.0210

[

KeyboardInterrupt: 

## 전체 데이터로 최종 모델 재학습

최적 파라미터로 전체 데이터셋 재학습

In [8]:
print("\n" + "="*60)
print("Retraining BiVAECF on full dataset...")
print("="*60)

# Create full dataset
full_train_set = Dataset.from_uir(data.itertuples(index=False), seed=SEED)
print(f"Full dataset - Users: {full_train_set.num_users}, Items: {full_train_set.num_items}")

# Retrain with best parameters
final_model = BiVAECF(
    k=best_params['k'],
    encoder_structure=best_params['encoder_structure'],
    act_fn='tanh',
    likelihood=best_params['likelihood'],
    n_epochs=best_params['n_epochs'],
    batch_size=best_params['batch_size'],
    learning_rate=best_params['learning_rate'],
    beta_kl=best_params['beta_kl'],
    seed=SEED,
    verbose=True
)

final_model.fit(full_train_set)

print("\nFinal model training complete!")


Retraining BiVAECF on full dataset...
Full dataset - Users: 8482, Items: 6695


  0%|          | 0/150 [00:00<?, ?it/s]


Final model training complete!


## Cold-Start 처리 포함 예측 생성

In [9]:
print("\n" + "="*60)
print("Generating predictions...")
print("="*60)

# Get all unique users from training data
all_users = data[DEFAULT_USER_COL].unique()
print(f"Total users: {len(all_users)}")

# Calculate item popularity for cold-start fallback
item_popularity = data[DEFAULT_ITEM_COL].value_counts().head(TOP_K).index.tolist()
print(f"Popular items for cold-start: {item_popularity[:5]}")

# Generate predictions
predictions = []
cold_start_count = 0

for user_id in all_users:
    if user_id in full_train_set.uid_map:
        try:
            recommended_items = final_model.recommend(user_id, k=TOP_K)
            
            # Ensure we have exactly TOP_K recommendations
            if len(recommended_items) < TOP_K:
                # Pad with popular items
                for item in item_popularity:
                    if item not in recommended_items:
                        recommended_items.append(item)
                    if len(recommended_items) >= TOP_K:
                        break
            
            for item_id in recommended_items[:TOP_K]:
                predictions.append({
                    DEFAULT_USER_COL: user_id,
                    DEFAULT_ITEM_COL: item_id
                })
        except:
            # Cold-start fallback
            for item_id in item_popularity[:TOP_K]:
                predictions.append({
                    DEFAULT_USER_COL: user_id,
                    DEFAULT_ITEM_COL: item_id
                })
            cold_start_count += 1
    else:
        # Cold-start fallback
        for item_id in item_popularity[:TOP_K]:
            predictions.append({
                DEFAULT_USER_COL: user_id,
                DEFAULT_ITEM_COL: item_id
            })
        cold_start_count += 1

print(f"\nPredictions generated: {len(predictions)}")
print(f"Cold-start users: {cold_start_count}/{len(all_users)} ({cold_start_count/len(all_users)*100:.1f}%)")

# Verify predictions
submit_df = pd.DataFrame(predictions)
print(f"\nSubmission shape: {submit_df.shape}")
print(f"Expected: ({len(all_users) * TOP_K}, 2)")

# Check if all users have exactly TOP_K recommendations
user_counts = submit_df[DEFAULT_USER_COL].value_counts()
if (user_counts == TOP_K).all():
    print(f"✓ All users have exactly {TOP_K} recommendations")
else:
    print(f"✗ Warning: Some users have != {TOP_K} recommendations")
    print(user_counts[user_counts != TOP_K])


Generating predictions...
Total users: 8482
Popular items for cold-start: ['R03237', 'R01214', 'R00056', 'R00773', 'R00944']

Predictions generated: 42410
Cold-start users: 0/8482 (0.0%)

Submission shape: (42410, 2)
Expected: (42410, 2)
✓ All users have exactly 5 recommendations


## 제출 파일 저장

In [10]:
# Create output directory with current date
current_date = datetime.now().strftime('%Y-%m-%d')
output_dir = f'outputs/{current_date}'
os.makedirs(output_dir, exist_ok=True)

# Generate filename with full timestamp
timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
param_str = f"k{best_params['k']}_epochs{best_params['n_epochs']}"
filename = f"{output_dir}/submit_BiVAECF_{param_str}_{timestamp}.csv"

# Save
submit_df.to_csv(filename, index=False)

print("\n" + "="*60)
print("Submission file saved!")
print("="*60)
print(f"Filename: {filename}")
print(f"Shape: {submit_df.shape}")
print(f"\nBest parameters used:")
for k, v in best_params.items():
    print(f"  {k}: {v}")
print(f"\nValidation Recall@{TOP_K}: {best_recall:.4f}")
print(f"Expected Public LB: 0.02-0.03 (significantly lower due to validation-test gap)")
print("\n⚠️  Note: BiVAECF may underperform on 99.9% sparse data compared to EASE/BPR")


Submission file saved!
Filename: outputs/2025-11-20/submit_BiVAECF_k100_epochs150_20251120110511.csv
Shape: (42410, 2)

Best parameters used:
  k: 100
  encoder_structure: [100]
  act_fn: tanh
  likelihood: bern
  n_epochs: 150
  batch_size: 256
  learning_rate: 0.001
  beta_kl: 0.5

Validation Recall@5: 0.0472
Expected Public LB: 0.02-0.03 (significantly lower due to validation-test gap)

⚠️  Note: BiVAECF may underperform on 99.9% sparse data compared to EASE/BPR


## 성능 요약 (Performance Summary)

In [11]:
print("\n" + "="*60)
print("BiVAECF PERFORMANCE SUMMARY")
print("="*60)
print(f"\nModel: BiVAECF")
print(f"Mode: {TUNING_MODE.upper()}")
print(f"\nBest Parameters:")
for k, v in best_params.items():
    print(f"  {k}: {v}")
print(f"\nValidation Recall@{TOP_K}: {best_recall:.4f}")
print(f"Expected Public LB: 0.02-0.03")
print(f"Expected Private LB: 0.02-0.03")
print(f"\nSubmission file: {filename}")
print(f"Total predictions: {len(submit_df)}")
print(f"Cold-start users: {cold_start_count} ({cold_start_count/len(all_users)*100:.1f}%)")
print("\n" + "="*60)
print("Recommendation: Compare with EASE (0.08-0.10) and BPR (0.09-0.12)")
print("BiVAECF is expected to perform worse due to extreme data sparsity.")
print("="*60)


BiVAECF PERFORMANCE SUMMARY

Model: BiVAECF
Mode: FAST

Best Parameters:
  k: 100
  encoder_structure: [100]
  act_fn: tanh
  likelihood: bern
  n_epochs: 150
  batch_size: 256
  learning_rate: 0.001
  beta_kl: 0.5

Validation Recall@5: 0.0472
Expected Public LB: 0.02-0.03
Expected Private LB: 0.02-0.03

Submission file: outputs/2025-11-20/submit_BiVAECF_k100_epochs150_20251120110511.csv
Total predictions: 42410
Cold-start users: 0 (0.0%)

Recommendation: Compare with EASE (0.08-0.10) and BPR (0.09-0.12)
BiVAECF is expected to perform worse due to extreme data sparsity.
